In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

import numpy as np
# import torch
import matplotlib.pyplot as plt

import random
import math


import sklearn
from sklearn.cluster import KMeans

from gsdmm import MovieGroupProcess

import wordninja

import time
import json

# TRY THIS OUT
# https://github.com/nlptown/nlp-notebooks/blob/master/Simple%20Sentence%20Similarity.ipynb


from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

import re

import spacy
import spacy_sentence_bert

nlp = spacy.load("en_core_web_trf")
# nlp_transform = spacy_sentence_bert.load_model('en_roberta_large_nli_stsb_mean_tokens')

# nltk.download('punkt')
# nltk.download("stopwords")
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')


# links to check out
# https://techblog.assignar.com/how-to-use-bert-sentence-embedding-for-clustering-text/
# https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da
# https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1
# https://explosion.ai/blog/spacy-transformers
# https://towardsdatascience.com/installing-tensorflow-with-cuda-cudnn-and-gpu-support-on-windows-10-60693e46e781


In [2]:
file = open("json_data.json")

allprompts = json.load(file)

In [3]:
# obtain word bag

prop_nouns = []
start_words = []
prop_nouns_spec = []

prompt_corpus = []

def get_bag(docs, out_list, word_list):
    # out_list = []

    stop_words = set(stopwords.words("english") + ["etc", "'s", ""])

    for include in ["what", "where", "when", "why", "who", "how"]:
        stop_words.discard(include)
        

    for doc in docs:

        if len(doc[1]) > 0 and len(doc[1][0]) > 0:
            prompt_corpus.append(doc[1])
        
        # separated_words = word_tokenize(doc[1])
        
        tagged_list = nlp(doc[1])

        prop_nouns_spec.append([(X.text, X.label_) for X in tagged_list.ents])
        
        lemmatizer = WordNetLemmatizer()
        lemmatized_list = []

        alpha_regex = re.compile('^[a-zA-Z]+$')

        at_start = True

        prev_prop = False

        for wordtag in tagged_list:

            word = str(wordtag)

            if alpha_regex.match(word):
                if at_start:
                    start_words.append(word)
                    at_start = False

                # if len(word) > 1:
                #     if not consecutiveUpper(word):
                #         split_word = wordninja.split(word)

                #         # print(split_word, word)

                #         if len(split_word) > 1:
                #             word = (split_word[0], wordtag.pos_)

                #             for subword in split_word[1:]:
                #                 tagged_list.append(subword)

                if word not in stop_words:
                    if len(word) > 1:
                        pos = ""
                        if wordtag.pos_ == "PROPN":
                            if prev_prop:
                                prop_nouns[-1] = prop_nouns[-1] + " " + word
                            else:
                                prop_nouns.append(word)
                            prev_prop = True
                        else:
                            prev_prop= False
                        
                        if wordtag.pos_[-1] == "J":
                            pos = "a"
                        elif wordtag.pos_[-1] == "N":
                            pos = "n"
                        elif wordtag.pos_[0] == "V":
                            pos = "v"
                        elif wordtag.pos_[-1] == "V":
                            pos = "r"
                        else:
                            continue

                        lemmatized_list.append(lemmatizer.lemmatize(word, pos=pos))
                        # if lemmatizer.lemmatize(word, pos=pos) == "u":
                        #     print(word, doc)
            else:
                if word in ["?", ".", "!"]:
                    at_start = True
                continue

            word_list += lemmatized_list
            out_list.append(" ".join(lemmatized_list))

        
    return out_list, word_list

In [4]:
word_list = []
numPrompts = []
prompt_corpus_processed = []

for college in allprompts:
    numPrompts.append(len(college[1]))
    
    prompt_corpus_processed, word_list = get_bag(college[1], prompt_corpus_processed, word_list)
    

word_list = set(word_list)

# for word in set(prop_nouns):
#     if word.lower() in word_list:
#         # print(word)
#         word_list.remove(word)

C:\Users\viche\miniconda3\envs\scikit\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
prompt_vectors = []

# i = 0

# print(len(prompt_corpus))
# print(prompt_corpus)

for doc in nlp.pipe(prompt_corpus):
    transformed_prompt = doc._.trf_data.tensors[-1]
    if transformed_prompt.shape == (1,768):
        prompt_vectors.append(transformed_prompt)

prompt_matrix = np.stack(prompt_vectors, axis=1)


# prompt_vectors[i]

# print(allprompts[0][1])

# for doc in nlp.pipe(["some text", "some other text"]):
#     tokvecs = doc._.trf_data.tensors[-1]

# tokvecs.shape